# Joining / Merging and Concatenating data

Today we will explore the different ways in which we can combine two dataframes into one in Pandas

## Why do we care about this?

There are several use cases for wanting to combine two or more datasets into one:

- In many applications, the data is generated in separate files (e.g. events) and we will need to concatenate them to have a single dataframe with all the entities we care about
    - This is because splitting data meaningfully allows many data processing solutions to scale by splitting the data into many different processing computers (nodes)
    - In the example of web events, event collectors usually store one file per day, or even per hour
- In most applications, different types of data will naturally live in different systems, and we will need to join them to get a complete view of a single entity
    - Especially in modern software systems (also known as microservice architecture), every different entity is handled by a completely isolated piece of software with its own data storage
    - This is usually awesome for everyone, but running analytics on that data sucks
- Finally, many times we will build multiple derived columns from our data, and at some point we will need to merge/concatenate them all together



## First, an introduction

Today we will review two basic methods to combine dataframes:

- pd.concat: used when you need to stick multiple datasets of the same thing together
- pd.merge: used when you need to combine data from multiple entities to give a full picture of the entity you care about

In [0]:
# We start like always, loading the 2nd best library ever written
import pandas as pd

In [0]:
# Then we sneakily load the data, we will use it later but for now let's keep it handy
df_test = pd.read_csv(
  filepath_or_buffer='https://raw.githubusercontent.com/aaronmcdaid/P2---Data-Analytics-With-Python/master/Berlin/WS%202019/5.%20Datasets/titanic_dataset/test.csv',
)
df_train = pd.read_csv(
  filepath_or_buffer='https://raw.githubusercontent.com/aaronmcdaid/P2---Data-Analytics-With-Python/master/Berlin/WS%202019/5.%20Datasets/titanic_dataset/train.csv',
)
df_survived = pd.read_csv(
  filepath_or_buffer='https://raw.githubusercontent.com/aaronmcdaid/P2---Data-Analytics-With-Python/master/Berlin/WS%202019/5.%20Datasets/titanic_dataset/gender_submission.csv',
).sample(frac=1, random_state=1).reset_index(drop=True)



### Concat (and its little sibling append)
Concat is used primarily to stick two sets of data together. Think of it like copying and pasting cells on excel, with a bit more intelligence and software that won't crash on you and make you lose 6 hours of work\*

Use concat when you have:

- Multiple dataframes with the same type of data (e.g. events from different dates)
- Different columns of the same data, with a meaningful index and an easy 1-1 relationship between them
    - For example, if you have 30 measurements from 56 different sensors, you can concatenate the measurement of all the sensors if the index is the measurement number for all
    
_\*True story_

#### Example: Let's stitch all the titanic passenger data together
The titanic dataset was originally thought to be used in a train-test ML pipeline. 
But now, we want to have the full picture of who was on the boat.

Let's stick all the __passenger__ data together

In [0]:
df_test.tail()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [0]:
df_train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [0]:
df_survived.head() #This one looks funky, let's keep it for later

,PassengerId,Survived
0,1250,0
1,1056,0
2,909,0
3,959,0
4,896,1


In [0]:
len(df_test)

418

In [0]:
len(df_train)

891

In [0]:
# print(891 + 418)

len(pd.concat([df_test,df_train]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


1309

As you can see on the cell above the dataframe produced by `pd.concat([df_test,df_train])` has the numbers of rows of the train dataset + the one of the test dataset.

Let's have a look of what is returned.

In [0]:
print(df_test.columns)

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [0]:
print(df_test.columns.tolist())
print(df_train.columns.tolist())

['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


In [0]:
pd.concat([df_train, df_test])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450
...,...,...,...,...,...,...,...,...,...,...,...,...
413,NaN,NaN,S,8.0500,"Spector, Mr. Woolf",0,1305,3,male,0,NaN,A.5. 3236
414,39.0,C105,C,108.9000,"Oliva y Ocana, Dona. Fermina",0,1306,1,female,0,NaN,PC 17758
415,38.5,NaN,S,7.2500,"Saether, Mr. Simon Sivertsen",0,1307,3,male,0,NaN,SOTON/O.Q. 3101262
416,NaN,NaN,S,8.0500,"Ware, Mr. Frederick",0,1308,3,male,0,NaN,359309


The `concat` method however knows a few things:

It knows that all columns with the same name should be the same (and aligns them neatly), and it knows that, if one column is missing, it probably means that the data is missing and will just fill it with NaN's.

Since this is a train-test dataset, the df_train has one extra column, "Survived", which is the one it is intended to predict. We will work on that later, but for now, just know that concat is not that stupid and can actually figure some things out for you.

#### What is up with the index?
If we look at the index, we'll see that it is kinda weird.
Pandas knows it's stupid, and will not dare make assumptions on whether the index means something. It could be an ID for all it knows.

But we humans know better, and know that those integers on the leftmost column mean jack shit, so we can tell pandas to ignore them and give us an dataframe with a normal looking index.

Oh, and an index without duplicates. That will come in handy when we do almost anything.

In [0]:
df1=pd.DataFrame([1,2,3,4])
df2=pd.DataFrame([1,2,3,4])

In [0]:
pd.concat([df1, df2], ignore_index=True)

,0
0,1
1,2
2,3
3,4
4,1
5,2
6,3
7,4


In [0]:
pd.concat([df_test,df_train],ignore_index=True).tail()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
1304,27.0,NaN,S,13.00,"Montvila, Rev. Juozas",0,887,2,male,0,0.0,211536
1305,19.0,B42,S,30.00,"Graham, Miss. Margaret Edith",0,888,1,female,0,1.0,112053
1306,NaN,NaN,S,23.45,"Johnston, Miss. Catherine Helen ""Carrie""",2,889,3,female,1,0.0,W./C. 6607
1307,26.0,C148,C,30.00,"Behr, Mr. Karl Howell",0,890,1,male,0,1.0,111369
1308,32.0,NaN,Q,7.75,"Dooley, Mr. Patrick",0,891,3,male,0,0.0,370376


In [0]:
pd.concat([df_test,df_train],ignore_index=True).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


**Exercise**

- Concatenate the df_test dataframe to the df_train dataset
- Concatenate the resulting dataframe dataframe to the df_survived dataset
    - What happens when you choose axis=1?
    - Are you confident the `Survived` column represents the real outcome of the passenger?

In [0]:
(pd.concat([df_test,df_train]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,34.5,NaN,Q,7.8292,"Kelly, Mr. James",0,892,3,male,0,NaN,330911
1,47.0,NaN,S,7.0000,"Wilkes, Mrs. James (Ellen Needs)",0,893,3,female,1,NaN,363272
2,62.0,NaN,Q,9.6875,"Myles, Mr. Thomas Francis",0,894,2,male,0,NaN,240276
3,27.0,NaN,S,8.6625,"Wirz, Mr. Albert",0,895,3,male,0,NaN,315154
4,22.0,NaN,S,12.2875,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,896,3,female,1,NaN,3101298
...,...,...,...,...,...,...,...,...,...,...,...,...
886,27.0,NaN,S,13.0000,"Montvila, Rev. Juozas",0,887,2,male,0,0.0,211536
887,19.0,B42,S,30.0000,"Graham, Miss. Margaret Edith",0,888,1,female,0,1.0,112053
888,NaN,NaN,S,23.4500,"Johnston, Miss. Catherine Helen ""Carrie""",2,889,3,female,1,0.0,W./C. 6607
889,26.0,C148,C,30.0000,"Behr, Mr. Karl Howell",0,890,1,male,0,1.0,111369


In [0]:
# Do them here
df = pd.concat([df_test,df_train], ignore_index=True)

df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,34.5,NaN,Q,7.8292,"Kelly, Mr. James",0,892,3,male,0,NaN,330911
1,47.0,NaN,S,7.0000,"Wilkes, Mrs. James (Ellen Needs)",0,893,3,female,1,NaN,363272
2,62.0,NaN,Q,9.6875,"Myles, Mr. Thomas Francis",0,894,2,male,0,NaN,240276
3,27.0,NaN,S,8.6625,"Wirz, Mr. Albert",0,895,3,male,0,NaN,315154
4,22.0,NaN,S,12.2875,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,896,3,female,1,NaN,3101298


In [0]:
pd.concat([df,df_survived], axis =1)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,index,PassengerId,Survived
0,34.5,NaN,Q,7.8292,"Kelly, Mr. James",0,892,3,male,0,NaN,330911,0.0,1250.0,0.0
1,47.0,NaN,S,7.0000,"Wilkes, Mrs. James (Ellen Needs)",0,893,3,female,1,NaN,363272,1.0,1056.0,0.0
2,62.0,NaN,Q,9.6875,"Myles, Mr. Thomas Francis",0,894,2,male,0,NaN,240276,2.0,909.0,0.0
3,27.0,NaN,S,8.6625,"Wirz, Mr. Albert",0,895,3,male,0,NaN,315154,3.0,959.0,0.0
4,22.0,NaN,S,12.2875,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,896,3,female,1,NaN,3101298,4.0,896.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,27.0,NaN,S,13.0000,"Montvila, Rev. Juozas",0,887,2,male,0,0.0,211536,NaN,NaN,NaN
1305,19.0,B42,S,30.0000,"Graham, Miss. Margaret Edith",0,888,1,female,0,1.0,112053,NaN,NaN,NaN
1306,NaN,NaN,S,23.4500,"Johnston, Miss. Catherine Helen ""Carrie""",2,889,3,female,1,0.0,W./C. 6607,NaN,NaN,NaN
1307,26.0,C148,C,30.0000,"Behr, Mr. Karl Howell",0,890,1,male,0,1.0,111369,NaN,NaN,NaN


#### `append()` method

Another way of concatenating dataset into one is to use the **`append()`** method that is called directly on a dataframe.

In [0]:
df_test.append(df_train, ignore_index = True).tail()
pd.concat([df_test, df_train], ignore_index = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,34.5,NaN,Q,7.8292,"Kelly, Mr. James",0,892,3,male,0,NaN,330911
1,47.0,NaN,S,7.0000,"Wilkes, Mrs. James (Ellen Needs)",0,893,3,female,1,NaN,363272
2,62.0,NaN,Q,9.6875,"Myles, Mr. Thomas Francis",0,894,2,male,0,NaN,240276
3,27.0,NaN,S,8.6625,"Wirz, Mr. Albert",0,895,3,male,0,NaN,315154
4,22.0,NaN,S,12.2875,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,896,3,female,1,NaN,3101298
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,27.0,NaN,S,13.0000,"Montvila, Rev. Juozas",0,887,2,male,0,0.0,211536
1305,19.0,B42,S,30.0000,"Graham, Miss. Margaret Edith",0,888,1,female,0,1.0,112053
1306,NaN,NaN,S,23.4500,"Johnston, Miss. Catherine Helen ""Carrie""",2,889,3,female,1,0.0,W./C. 6607
1307,26.0,C148,C,30.0000,"Behr, Mr. Karl Howell",0,890,1,male,0,1.0,111369


It's essentially the same :)

### Merge (and its little sibling join)
Merge is used to comnbine different kinds of information on the same type of entity, coming from sources that don't necessarily are the same.

You use merge when you:

- Want to complement rows of your dataframe with extended information on some of its columns
    - E.g. if you had a new dataframe with data on which cabins in the titanic were less than 10 m from a life raft, and you want to add that field to each individual passenger
- Need to combine dataframes where the rows on each dataframe have different meanings:
    - E.g. one dataframe is transactions and the other is product information like price and name



### Example: Did the passengers survive?

We have the `df_survived` dataset that tells us who survived, but it's completely mixed up. Let's join it properly!

In [0]:
df = pd.concat([df_train, df_test])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [0]:
df.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450


In [0]:
df.merge(
    df_survived, 
    on='PassengerId'
).head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived_x,Ticket,index,Survived_y
0,34.5,NaN,Q,7.8292,"Kelly, Mr. James",0,892,3,male,0,NaN,330911,167,0
1,47.0,NaN,S,7.0000,"Wilkes, Mrs. James (Ellen Needs)",0,893,3,female,1,NaN,363272,383,1
2,62.0,NaN,Q,9.6875,"Myles, Mr. Thomas Francis",0,894,2,male,0,NaN,240276,325,0
3,27.0,NaN,S,8.6625,"Wirz, Mr. Albert",0,895,3,male,0,NaN,315154,371,0
4,22.0,NaN,S,12.2875,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,896,3,female,1,NaN,3101298,4,1


#### Suffixes will let us easily identify where a column came from

In [0]:
df.merge(
    df_survived, 
    on='PassengerId', 
    suffixes=('_df', '_df_s')
)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived_df,Ticket,index,Survived_df_s
0,34.5,NaN,Q,7.8292,"Kelly, Mr. James",0,892,3,male,0,NaN,330911,167,0
1,47.0,NaN,S,7.0000,"Wilkes, Mrs. James (Ellen Needs)",0,893,3,female,1,NaN,363272,383,1
2,62.0,NaN,Q,9.6875,"Myles, Mr. Thomas Francis",0,894,2,male,0,NaN,240276,325,0
3,27.0,NaN,S,8.6625,"Wirz, Mr. Albert",0,895,3,male,0,NaN,315154,371,0
4,22.0,NaN,S,12.2875,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,896,3,female,1,NaN,3101298,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,NaN,NaN,S,8.0500,"Spector, Mr. Woolf",0,1305,3,male,0,NaN,A.5. 3236,233,0
414,39.0,C105,C,108.9000,"Oliva y Ocana, Dona. Fermina",0,1306,1,female,0,NaN,PC 17758,77,1
415,38.5,NaN,S,7.2500,"Saether, Mr. Simon Sivertsen",0,1307,3,male,0,NaN,SOTON/O.Q. 3101262,255,0
416,NaN,NaN,S,8.0500,"Ware, Mr. Frederick",0,1308,3,male,0,NaN,359309,150,0


In [0]:
df_full = df.merge(df_survived, on='PassengerId', suffixes=('_df', '_df_s'))

In [0]:
len(df_full)

418

In [0]:
len(df)

1309

### What happened!?!?

Merge automatically runs an `inner` join between the two datasets. Meaning that only the rows where a record was found on __both__ will be in the output.

In this case, we only really care about the passengers, and adding as much data as we can to what we know of them, so instead of running the default `inner` join, we will want to run a `left join`.

Wat!? – We will explain, but if you want to play around and understand the differences, go here: https://drive.google.com/open?id=15ckU7_DOFoxA5UJe0BeyFcc8pE8lxMhY

In [0]:
df_full = df.merge(
    df_survived, 
    on='PassengerId', 
    suffixes=('_df', '_df_s'),
    how='left'
)

len(df_full)

1309

In [0]:
df_full.head(5)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived_df,Ticket,index,Survived_df_s
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,NaN,NaN
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,NaN,NaN
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,NaN,NaN
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,NaN,NaN
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,NaN,NaN


**Exercises**
* Perform an inner join of the `df_test` and the `df_survived` dataset on the `PassengerId` column


In [0]:
df_test.merge(df_survived, on='PassengerId')

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,index,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,167,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,383,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,325,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,371,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,233,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,77,1
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,255,0
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,150,0


In [0]:
# Do it here

In [0]:
# Do it here

#### Join on multiple values

As with groupby, it is also possible to merge dataframes on not just one column but multiple columns by passing a list of columns we want to perform the merge on to the `on` parameter.

In [0]:
df_test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [0]:
Age = df_test.groupby(['Sex','Pclass']).Age.mean().reset_index()
Age 

,Sex,Pclass,Age
0,female,1,41.333333
1,female,2,24.376552
2,female,3,23.073400
3,male,1,40.520000
4,male,2,30.940678
5,male,3,24.525104


In [0]:
Sib = df_test.groupby(['Sex','Pclass']).SibSp.mean().reset_index()
Sib

,Sex,Pclass,SibSp
0,female,1,0.560000
1,female,2,0.533333
2,female,3,0.583333
3,male,1,0.403509
4,male,2,0.301587
5,male,3,0.404110


In [0]:
Age.merge(Sib, how = 'inner', on = ['Sex','Pclass'])

,Sex,Pclass,Age,SibSp
0,female,1,41.333333,0.560000
1,female,2,24.376552,0.533333
2,female,3,23.073400,0.583333
3,male,1,40.520000,0.403509
4,male,2,30.940678,0.301587
5,male,3,24.525104,0.404110


#### left_on and right_on parameters

The `on`parameter is used when the key has the same name in both dataframes. Sometimes, however, the key can have a different name in each of the dataframes. This is when the following parameters are useful:
 
- **left_on**: the name of the key in the left dataframe
- **right_on**: the name of the key in the right dataframe
 

In [0]:
df_survived.reset_index(inplace=True)

In [0]:
df.merge(df_survived, left_on='PassengerId', right_index=True)

,PassengerId,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId_x,Pclass,Sex,SibSp,Survived_x,Ticket,level_0,index,PassengerId_y,Survived_y
0,1,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,1,1,1056,0
1,2,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,2,2,909,0
2,3,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,3,3,959,0
3,4,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,4,4,896,1
4,5,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,5,5,1269,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,413,33.0,C78,Q,90.0000,"Minahan, Miss. Daisy E",0,413,1,female,1,1.0,19928,413,413,1147,0
413,414,NaN,NaN,S,0.0000,"Cunningham, Mr. Alfred Fleming",0,414,2,male,0,0.0,239853,414,414,964,1
414,415,44.0,NaN,S,7.9250,"Sundman, Mr. Johan Julian",0,415,3,male,0,1.0,STON/O 2. 3101269,415,415,1288,0
415,416,NaN,NaN,S,8.0500,"Meek, Mrs. Thomas (Annie Louise Rowley)",0,416,3,female,0,0.0,343095,416,416,1127,0


**Exercise**

Perform an inner join on the **Age** and  **Parent** dataframes (as defined below) on the `Sex` column
  * Extract the average age of the Titanic passengers given their Sex, class and Embarked port and save it into a new dataframe called **Age**
  * Extract the median number of ParCh of the Titanic passengers given their Sex, class and Embarked port and save it into a new dataframe called **Parent**


In [0]:
Age = df.groupby(['Sex','Pclass','Embarked']).mean().Age.reset_index()

# age = df.groupby(['Sex', 'Pclass', 'Embarked']).mean().Age.reset_index()
# parent = df.groupby(['Sex', 'Pclass', 'Embarked']).median().Parch.reset_index()

In [0]:
Age

,Sex,Pclass,Embarked,Age
0,female,1,C,38.107692
1,female,1,Q,35.000000
2,female,1,S,35.609375
3,female,2,C,19.363636
4,female,2,Q,30.000000
5,female,2,S,28.455165
6,female,3,C,16.818182
7,female,3,Q,24.333333
8,female,3,S,22.854771
9,male,1,C,40.047619


In [0]:
# Extract the median number of ParCh of the Titanic passengers given their Sex, class and Embarked port 
# and save it into a new dataframe called Parent
Parent = df.groupby(['Sex','Pclass','Embarked']).median()['Parch'].reset_index()

In [0]:
Parent

,Sex,Pclass,Embarked,Parch
0,female,1,C,0.0
1,female,1,Q,0.0
2,female,1,S,0.0
3,female,2,C,0.0
4,female,2,Q,0.0
5,female,2,S,0.0
6,female,3,C,1.0
7,female,3,Q,0.0
8,female,3,S,0.0
9,male,1,C,0.0


In [0]:
Age.merge(Parent, how='inner', on = ['Sex','Pclass','Embarked'], suffixes=('Age','Parent'))
# inner join
# age.merge(parent, how = 'inner', on = ['Sex', 'Pclass', 'Embarked'])

,Sex,Pclass,Embarked,Age,Parch
0,female,1,C,38.107692,0.0
1,female,1,Q,35.000000,0.0
2,female,1,S,35.609375,0.0
3,female,2,C,19.363636,0.0
4,female,2,Q,30.000000,0.0
5,female,2,S,28.455165,0.0
6,female,3,C,16.818182,1.0
7,female,3,Q,24.333333,0.0
8,female,3,S,22.854771,0.0
9,male,1,C,40.047619,0.0


### Excercise / Homework: How far is this passenger from the mean of their group?

Joins allow us to do a very powerful type of analysis on the titanic dataset. Let's use what we learned in the previous class to create some group averages, and then see how the passengers fall with regards to their peers.


Follow these steps:

- Concatenate df_train and df_test (if not done already)
- Get the average AGE and FARE by Sex and Pclass
- Join the base dataset to the dataframe created in the previous step, to get the average AGE and FARE of each passenger's Sex and Pclass
- Get the difference between the passenger's AGE and FARE from the average of their Sex and Pclass

In [0]:
# Do it here